# Notes
* 获取senti words
* 别忘了clean里面的符号（用clean代替一些代码）
* 拿了大概五百个
* filter weights用sent2net里的词汇
* 从sentiwordnet,如何从文件里抓取极性词，分数大于0.8，建立两个得到list的情感词。分开positive和negative
* 加到一个list
* layer = return true

* 自己得到cnn_filter weights
* 默认train的时候得到weights是initilizer randomly的
* 自己train的时候，trainable为false，因为已经得到情感词
* weight.shape（window_size,feature_dimension(embedding),number of words/number of filter）
* bias.shape(number of sentiment words/number of filter)一维list 
* 两个合在一起

## CNN
* function:get the sentiment words,use the filters to get
* input:()
* filter:to extract some features
* filter_length:the quantity of filters
* max_pooling:
* output:(batch_size*)
* 用两个CNN，一个是knowledge的，如果没找完，就要用random了

## LSTM
* function:
* input:
* output:
* 

# questions
* 为什么maxpooling得到这个
* 可以试试别的
* filter的作用

#### Test
* 两个最重要的是vocab2int 和 embedding matrix
* 纯CNN跑，用全部的数据，不用分句
* 后面三个是以句子为单位
* cnn layer2是基于知识的
* 用skLearn得到recall 和precision


In [1]:
import pickle
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.embeddings import Embedding
from keras.layers import Convolution1D, MaxPooling1D, GlobalMaxPooling1D, Input, Dense, Reshape, LSTM, GRU
from keras.layers.merge import concatenate, add
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import re
import gensim
import numpy as np
from util.util_functions import getWordIdx
import nltk
import string
from sklearn.metrics import classification_report, roc_auc_score

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#loading the train_copus_padded data from .pickle file
file = open('pickle_data/train_copus_pad.pickle','rb')
train_copus_padded = pickle.load(file)

file = open('pickle_data/test_copus_pad.pickle','rb')
test_copus_padded = pickle.load(file)

file = open('pickle_data/vocab_train.pickle','rb')
vocab_to_int_train = pickle.load(file)

file = open('pickle_data/embedding_matrix','rb')
embedding_matrix = pickle.load(file)

file = open('pickle_data/train_label.pickle','rb')
train_label = pickle.load(file)

file = open('pickle_data/test_label.pickle','rb')
test_label = pickle.load(file)
file.close()

In [3]:
print('train test data shape:',train_copus_padded.shape, test_copus_padded.shape)
print('embedding_matrix shape:', embedding_matrix.shape)
#the size of vocabulary
vocab_size = len(vocab_to_int_train)
print('vocabulary size:', vocab_size)
# the maximal length of every sentence
maxlen_sent = train_copus_padded.shape[1]
maxlen_word = train_copus_padded.shape[2]
print('max sent length:', maxlen_sent, '\nmax word length:', maxlen_word)

train test data shape: (25000, 36, 224) (25000, 36, 224)
embedding_matrix shape: (97162, 300)
vocabulary size: 97162
max sent length: 36 
max word length: 224


## Extracting sentiment words from SentiWordNet

In [ ]:
file_path = 'resource/SentiWordNet_3.0.0_20130122.txt'
content = []
with open(file_path,'r',encoding='UTF-8') as f:
    while(1):
        lines = f.readline()
        if(lines[0] is ('a' or 'n' or 'v' or 'r')):
            content.append(lines.split('\t'))
        elif(lines[0] is '#'):
            continue
        else:
            break
             
#get the sentiment words(score>0.8)
neg_words1 = []
pos_words1 = []
for lines in content:
    if(float(lines[2])>=0.8):
        pos_words1.append(lines[4])
    elif(float(lines[3])>=0.8):
        neg_words1.append(lines[4])

#get the positive sentiment words
#seperate the multiple words
pos_words2 = []
for i in range(len(pos_words1)):
    pos_words2.append(pos_words1[i].split(' '))

#get the one dimension elements
pos_words3 = []
for i in pos_words2:
    for j in i:
        pos_words3.append(j)

#delete the '#4'
pos_words4 = []
for elem in pos_words3:
    pos_words4.append(re.sub('#\d','',elem))
    
#delete the 'number'
pos_lexicon = []
for elem in pos_words4:
    pos_lexicon.append(re.sub('\d','',elem))

    
#get the negative sentiment words
#seperate the multiple words
neg_words2 = []
for i in range(len(neg_words1)):
    neg_words2.append(neg_words1[i].split(' '))

#get the one dimension elements
neg_words3 = []
for i in neg_words2:
    for j in i:
        neg_words3.append(j)

#delete the '#4'
neg_words4 = []
for elem in neg_words3:
    neg_words4.append(re.sub('#\d','',elem))
    
#delete the 'number'
neg_lexicon = []
for elem in neg_words4:
    neg_lexicon.append(re.sub('\d','',elem))

#merge the positive and negative words
senti_lexicon = pos_lexicon + neg_lexicon
#shuffule the these lists
from sklearn.utils import shuffle 
senti_lexicon = shuffle(senti_lexicon,random_state = 0)

In [13]:
len(senti_lexicon)

533

In [6]:
#map the sentiment words to integer based on vocab2int
senti2int = [getWordIdx(word, vocab_to_int_train) for word in senti_lexicon]

#get the filter weights based on the sentiment words&vocab2int&embedding_matrix
def Find_Filter_Weight(senti2int):
    """sentiwords is the list"""
    word_filter_weights = []
    bias_weights = []
    filter_len = 1
    for i in senti2int:
        vector = embedding_matrix[i]  # shape: 300
        vector = np.expand_dims(vector, axis=0) #shape: 1x 300
        vector = np.expand_dims(vector, axis=2) #shape: 1x 300 x 1
        if len(word_filter_weights) == 0:
            word_filter_weights = vector
        else:
            word_filter_weights = np.concatenate((word_filter_weights, vector), axis=2)   
    #shape is (1, 300, 533)
    
    bias_weights = np.zeros(len(senti2int))
    cnn_wordfilter_weights = [word_filter_weights, bias_weights]
    
    return cnn_wordfilter_weights    

In [7]:
CNN_weights = Find_Filter_Weight(senti2int)
CNN_weights[0].shape

(1, 300, 533)

## Experiment 
* record the result of running the different vesions
* change the parameter to run and record the result again

### data prepocessing for CNN/GRU/LSTM

In [8]:
#define all of the functions
punctuation_list = list(string.punctuation)

def word_tokenize(sent):
    tokenized_text = nltk.word_tokenize(sent)  # this gives you a list of words
    tokenized_text = [token.lower() for token in tokenized_text if token not in punctuation_list]  # optional: convert all words to lower case
    return tokenized_text

def readfile(filename):
    with open(filename,'r',encoding='UTF-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    #strip()读出有效文件，形成一个list
    #split()读成有效文件，根据一行来形成一个list
    return content

In [9]:
#**********loading data
#get the movie review "list of string"
test_neg = readfile('data/aclImdb/test-neg.txt')
test_pos = readfile('data/aclImdb/test-pos.txt')
train_neg = readfile('data/aclImdb/train-neg.txt')
train_pos = readfile('data/aclImdb/train-pos.txt')

#use these lists to label the movie reviews
test_neg_label = [0 for i in range(len(test_neg))]
test_pos_label = [1 for i in range(len(test_pos))]
train_neg_label =[0 for i in range(len(train_neg))]
train_pos_label =[1 for i in range(len(train_pos))]


#merge the test label
test_label = test_neg_label + test_pos_label

#merge the train label
train_label = train_neg_label + train_pos_label

#merge the test data
test_data = test_neg + test_pos

#merge the train data
train_data = train_neg + train_pos

#shuffule the these lists
from sklearn.utils import shuffle 
train_data , train_label = shuffle(train_data , train_label , random_state = 0)
test_data , test_label = shuffle(test_data ,test_label , random_state = 0) 

In [10]:
#preprocessing the data for the cnn layer
#word tokenizing the data
training_data = []
for data in train_data:
    training_data.append(word_tokenize(data))

testing_data = []
for data in test_data:
    testing_data.append(word_tokenize(data))
    
#map the word to integer
x_train = []
word2int = []
for review in training_data:
    word2int =[getWordIdx(word, vocab_to_int_train) for word in review]
    x_train.append(word2int)
    
x_test = []
word2int = []
for review in testing_data:
    word2int = [getWordIdx(word, vocab_to_int_train) for word in review]
    x_test.append(word2int)


#show and padding the data
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
sequence = keras.preprocessing.sequence
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_word)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen_word)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

### CNN Model

In [29]:
#build the model
num_filter = 100
batch_size = 100
num_epoch = 5

input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = Convolution1D(nb_filter=num_filter,
                            filter_length=3,
                            border_mode='same',
                            activation='tanh',
                            subsample_length=1
                            )(embeded)
cnn_out = GlobalMaxPooling1D()(hidden_layer)
dense = Dense(50, activation='sigmoid')(cnn_out)
#dense = Dense(dim, activation='softmax')(cnn_out)
Final = Dense(1,activation = 'sigmoid')(dense)#为什么换成softmax就很差
model = Model(inputs=input_data, outputs=[Final])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

#training the model
print('Train...')
model.fit(x_train, train_label,
          batch_size=batch_size,
          epochs=num_epoch,
          validation_data=(x_test, test_label))
score, acc = model.evaluate(x_test, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)
#the validate is too much?


C:\Users\pc\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=100, activation="tanh", padding="same", kernel_size=3, strides=1)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 224, 300)          30089100  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 224, 100)          90100     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 51        
Total params: 30,184,301
Trainable params: 95,201
Non-trainable params: 30,089,100
___________________________________________________________

In [30]:
pred_test_prob = model.predict(x_test, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    pred_test = (pred_test_prob>0.5).astype('int32')
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))

print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1],digits=4))

25000/25000 [==============================] - 44s 2ms/step
Accuracy: 0.8800
             precision    recall  f1-score   support

          0     0.9221    0.8302    0.8738     12500
          1     0.8456    0.9298    0.8857     12500

avg / total     0.8838    0.8800    0.8797     25000



### classical LSTM model

In [17]:
#build LSTM model
batch_size = 100
num_epoch = 5

input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
# hidden_layer = LSTM(128, activation = 'sigmoid',dropout=0.2, recurrent_dropout=0.2)(embeded)
#hidden_layer = LSTM(128, activation = 'relu',dropout=0.2, recurrent_dropout=0.2)(embeded)
# hidden_layer = LSTM(128, activation = 'tanh',dropout=0.1, recurrent_dropout=0.2)(embeded)
hidden_layer = LSTM(128, activation = 'tanh',dropout=0.2, recurrent_dropout=0.2)(embeded)
dense = Dense(50,activation = 'sigmoid')(hidden_layer)
Final = Dense(1,activation = 'sigmoid')(dense)

#compile the model
model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 224, 300)          30089100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_3 (Dense)              (None, 50)                6450      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 30,315,249
Trainable params: 226,149
Non-trainable params: 30,089,100
_________________________________________________________________


In [18]:
#training the model
print('Train...')
model.fit(x_train, train_label,
          batch_size=batch_size,
          epochs=num_epoch,
          validation_data=(x_test, test_label))
score, acc = model.evaluate(x_test, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 461s 18ms/step - loss: 0.5767 - acc: 0.6963 - val_loss: 0.4569 - val_acc: 0.8012
Epoch 2/5
25000/25000 [==============================] - 428s 17ms/step - loss: 0.5340 - acc: 0.7453 - val_loss: 0.5031 - val_acc: 0.7663
Epoch 3/5
25000/25000 [==============================] - 436s 17ms/step - loss: 0.5433 - acc: 0.7238 - val_loss: 0.4176 - val_acc: 0.8129
Epoch 4/5
25000/25000 [==============================] - 436s 17ms/step - loss: 0.4518 - acc: 0.7963 - val_loss: 0.3764 - val_acc: 0.8390
Epoch 5/5
25000/25000 [==============================] - 111s 4ms/step
Test accuracy: 0.8545199971199036


In [23]:
#measure the model
pred_test_prob = model.predict(x_test, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    
    pred_test = (pred_test_prob>0.5).astype('int32')
    
    
    
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))



print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1],digits=4))

25000/25000 [==============================] - 111s 4ms/step
Accuracy: 0.8545
             precision    recall  f1-score   support

          0     0.8756    0.8264    0.8503     12500
          1     0.8356    0.8826    0.8585     12500

avg / total     0.8556    0.8545    0.8544     25000



### classical GRU model

In [24]:
#build GRU model
batch_size = 100
num_epoch = 5

input_data = Input(shape=(maxlen_word,))
embeded = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)(input_data)
hidden_layer = GRU(128, activation = 'tanh',dropout=0.2, recurrent_dropout=0.2)(embeded)
dense = Dense(50,activation = 'sigmoid')(hidden_layer)
Final = Dense(1,activation = 'sigmoid')(dense)

#compile the model
model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 224, 300)          30089100  
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               164736    
_________________________________________________________________
dense_5 (Dense)              (None, 50)                6450      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 30,260,337
Trainable params: 171,237
Non-trainable params: 30,089,100
_________________________________________________________________


In [25]:
#training the model
print('Train...')
model.fit(x_train, train_label,
          batch_size=batch_size,
          epochs=num_epoch,
          validation_data=(x_test, test_label))
score, acc = model.evaluate(x_test, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 339s 14ms/step - loss: 0.5899 - acc: 0.6816 - val_loss: 0.4809 - val_acc: 0.7717
Epoch 2/5
25000/25000 [==============================] - 326s 13ms/step - loss: 0.4704 - acc: 0.7861 - val_loss: 0.4959 - val_acc: 0.7667
Epoch 3/5
25000/25000 [==============================] - 331s 13ms/step - loss: 0.4066 - acc: 0.8232 - val_loss: 0.3409 - val_acc: 0.8564
Epoch 4/5
25000/25000 [==============================] - 327s 13ms/step - loss: 0.3573 - acc: 0.8480 - val_loss: 0.3231 - val_acc: 0.8612
Epoch 5/5
25000/25000 [==============================] - 94s 4ms/step
Test accuracy: 0.8765200004577637


In [26]:
#measure the model
pred_test_prob = model.predict(x_test, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    pred_test = (pred_test_prob>0.5).astype('int32')
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))

print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1],digits=4))

25000/25000 [==============================] - 95s 4ms/step
Accuracy: 0.8765
             precision    recall  f1-score   support

          0     0.8963    0.8516    0.8734     12500
          1     0.8586    0.9014    0.8795     12500

avg / total     0.8775    0.8765    0.8764     25000



### KCNN+GRU(The document is seperated by sentence first and then by word)

In [ ]:
#build the kCNN+GRU model
def slice(x, index):
    """ Define a tensor slice function
    
    """
    return x[:, index, :]



input_data = Input(shape = (maxlen_sent,maxlen_word))
embedding_layer = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)
cnn_layer1 = Convolution1D(nb_filter=50,
                            filter_length=3,
                            border_mode='same',
                            activation='relu',
                            subsample_length=1)

cnn_layer2 = Convolution1D(nb_filter=CNN_weights[0].shape[2],
                            filter_length=1,
                            border_mode='same',
                            activation='relu',
                           weights = CNN_weights,
                           trainable = False,
                            subsample_length=1)

#embedding matrix shape[1]是300，每个vector的维度
max_pooling_layer = GlobalMaxPooling1D()

#************
stack_layer = Lambda(lambda x: K.stack(x, axis=1))

# interate through sentences in a document
cnn_out = []
for i in range(maxlen_sent):
    #以每个影评的每个句子为输入
    sent = Lambda(slice, arguments={'index': i,})(input_data)#use the lambda to enclose the function as the slicing layer
    sent_embedding = embedding_layer(sent)#input shape:(padded_sentence_number),output shape:(nb_words_padded,dimension)
    
    sent_cnn1 = cnn_layer1(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    # we use standard max over time pooling
    sent_cnn1 = max_pooling_layer(sent_cnn1)  # output shape: (None, nb_filter)
    
    sent_cnn2 = cnn_layer2(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    # we use standard max over time pooling
    sent_cnn2 = max_pooling_layer(sent_cnn2)  # output shape: (None, nb_filter)
    
    sent_cnn = concatenate([sent_cnn1, sent_cnn2])
    
    cnn_out.append(sent_cnn)
cnn_out = stack_layer(cnn_out)  # out shape: (None, maxlen_sent, nb_filter)



#处理句子，从头到尾
gru= GRU(256, dropout=0.3, recurrent_dropout=0.2)(cnn_out)
dense1 = Dense(128, activation='sigmoid')(gru)
Final = Dense(1, activation='sigmoid')(dense1)

model = Model(inputs=[input_data], outputs=[Final])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
#training the model
batch_size = 300
num_epoch = 5

print('Training...')
model.fit(train_copus_padded, train_label,
          batch_size=batch_size,
          epochs=num_epoch,
          validation_data=(test_copus_padded, test_label))

score, acc = model.evaluate(test_copus_padded, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

In [ ]:
#measure the model
pred_test_prob = model.predict(test_copus_padded, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    pred_test = (pred_test_prob>0.5).astype('int32')
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))

print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1]))




### GRU+CNN

In [4]:
#build the GRU+CNN model
batch_size = 100
num_epoch = 5

def slice(x, index):
    """ Define a tensor slice function
    """
    return x[:, index, :]



input_data = Input(shape = (maxlen_sent,maxlen_word), dtype='int32', name='input_data')
embedding_layer = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)
gru_layer = GRU(128, activation = 'tanh',dropout=0.2, recurrent_dropout=0.2)#when return_sequences=true,the lstm model returns the outputs of every timestep
cnn_layer = Convolution1D(nb_filter=100,
                            filter_length=3,
                            border_mode='same',
                            activation='tanh',
                            subsample_length=1)

#embedding matrix shape[1]是300，每个vector的维度
max_pooling_layer = GlobalMaxPooling1D()

#************
stack_layer = Lambda(lambda x: K.stack(x, axis=1))

# interate through sentences in a document
gru_out = []
for i in range(maxlen_sent):
    #以每个影评的每个句子为输入
    sent = Lambda(slice, arguments={'index': i,})(input_data)#use the lambda to enclose the function as the slicing layer
    sent_embedding = embedding_layer(sent)#input shape:(padded_sentence_number),output shape:(nb_words_padded,dimension)
    sent_gru = gru_layer(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    gru_out.append(sent_gru)
gru_out = stack_layer(gru_out)  # out shape: (None, maxlen_sent, nb_filter)


#
#处理句子，从头到尾
sent_cnn = cnn_layer(gru_out)
cnn_out = max_pooling_layer(sent_cnn)
dense1 = Dense(50,activation='sigmoid')(cnn_out)
dense2 = Dense(1, activation='sigmoid')(dense1)

model = Model(inputs=[input_data], outputs=[dense2])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=100, kernel_size=3, strides=1, padding="same")`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 36, 224)      0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224)          0           input_data[0][0]                 
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224)          0           input_data[0][0]                 
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 224)          0           input_data[0][0]                 
__________________________________________________________________________________________________
lambda_5 (

In [ ]:
#training the model
print('Training...')
model.fit(train_copus_padded, train_label,
          batch_size=batch_size,
          epochs=5,
          validation_data=(test_copus_padded, test_label))

score, acc = model.evaluate(test_copus_padded, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
  800/25000 [..............................] - ETA: 5:10:41 - loss: 0.7318 - acc: 0.5113

In [ ]:
#measure the model
pred_test_prob = model.predict(x_test, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    pred_test = (pred_test_prob>0.5).astype('int32')
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))

print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1]))


### CNN+GRU model

In [ ]:
#build the CNN+GRU model
Bidirectional = keras.layers.Bidirectional
def slice(x, index):
    """ Define a tensor slice function
    
    """
    return x[:, index, :]



input_data = Input(shape = (maxlen_sent,maxlen_word))
embedding_layer = Embedding(vocab_size, embedding_matrix.shape[1], input_length=maxlen_word, 
                            weights=[embedding_matrix], trainable=False)
cnn_layer = Convolution1D(nb_filter=100,
                            filter_length=3,
                            border_mode='same',
                            activation='tanh',
                            subsample_length=1)

#embedding matrix shape[1]是300，每个vector的维度
max_pooling_layer = GlobalMaxPooling1D()

#************
stack_layer = Lambda(lambda x: K.stack(x, axis=1))

# interate through sentences in a document
cnn_out = []
for i in range(maxlen_sent):
    #以每个影评的每个句子为输入
    sent = Lambda(slice, arguments={'index': i,})(input_data)#use the lambda to enclose the function as the slicing layer
    sent_embedding = embedding_layer(sent)#input shape:(padded_sentence_number),output shape:(nb_words_padded,dimension)
    
    sent_cnn = cnn_layer(sent_embedding) # output shape: (None, maxlen_word, nb_filter)
    # we use standard max over time pooling
    sent_cnn = max_pooling_layer(sent_cnn)  # output shape: (None, nb_filter)
    
    cnn_out.append(sent_cnn)
cnn_out = stack_layer(cnn_out)  # out shape: (None, maxlen_sent, nb_filter)



#处理句子，从头到尾
hidden_layer_one = GRU(128, activation='tanh',dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(cnn_out)
hidden_layer_two = Bidirectional(GRU(128, activation='tanh',dropout=0.2, recurrent_dropout=0.2))(hidden_layer_one)
dense1 = Dense(64, activation='sigmoid')(hidden_layer_two)
dense2 = Dense(1,activation='sigmoid')(dense1)

model = Model(inputs=[input_data], outputs=[dense2])
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

C:\Users\pc\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=100, activation="tanh", padding="same", kernel_size=3, strides=1)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 36, 224)      0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224)          0           input_6[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224)          0           input_6[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 224)          0           input_6[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [ ]:
#training the model
batch_size = 100
epoch_num = 5

print('Training...')
model.fit(train_copus_padded, train_label,
          batch_size=batch_size,
          epochs=epoch_num,
          validation_data=(test_copus_padded, test_label))

score, acc = model.evaluate(test_copus_padded, test_label,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
17500/25000 [====================>.........] - ETA: 21:35 - loss: 0.4665 - acc: 0.7631

In [ ]:
#measure the model
pred_test_prob = model.predict(test_copus_padded, batch_size=batch_size, verbose=True)
# predict the class label
if pred_test_prob.shape[-1]>1:
    pred_test = pred_test_prob.argmax(axis=-1)
else:
    pred_test = (pred_test_prob>0.5).astype('int32')
    pred_test = pred_test.reshape(pred_test.shape[0])

acc = np.sum(pred_test == test_label) / float(len(test_label))

print("Accuracy: %.4f" % (acc))   

print(classification_report(test_label, pred_test, labels=[0, 1],digits=4))

### paper
* 单纯用cnn只能抓到n gram的信息，没办法拿到句子和句子的semantic relation
* 单纯用rnn同理，rnn 处理long term dependency有局限，导致前面的内容没办法没model所抓取。
* GRU>LSTM，后面的组合用gru
* 不分句没办法抓取semantic relation,reference document paper,记录为什么要分句（层级word(hieratical)）
* cnn+gru(cnn先抓取句子中重要的sentiment feature,rnn抓取每个句子之间的关系，最后输出feature vector)
* gru+cnn(gru先抓取句子中的词语之间的序列关系，cnn再抓取重要的句子(gru_output)，但是不能拿到句子和句子之间的semantic relation，结果较差)
* Kcnn+gru(加入eternel knowledge(extracting sentiword from SentiWordNet)，为了提升cnn+gru中cnn抓取sentiment word的能力，)，reference 老师paper
* cnn+gru单纯based on data，所以model需要大量的high quality labelled data去学习（局限），会出现overfitting(training data少，或者质量不好)
* KCNN+gru加入human knowledge，construct sentiment word filters for cnn，更有效地提取sentiment Word,minimize reliance on training data
* rule-based的方法很难建立，其次在改的时候也需要改一连串的规则
* DL语言的复杂性造成data很多，很难获取
* DL的overfitting
* NLP tookits, POS tagger(词性)
* 实验调整filter 10~100

### how to write paper
* 阅读paper
* abstract+conclusion
* 归类（based on data(传统ML for sentiment analysis,DL_NN for sentiment analysis,cnn/rnn/combination of cnn+rnn）,+externel knowledge,sentimentWordNet)),记录每个method的局限。ML 需要复杂的feature engineering，需要externel NLP toolkits（不完美）,会出现error，影响model，DL 过于依赖大量label data，记录externel knowledge method如何使用sentiwordnet,大部分可能是使用pattern matching,our model 优势把sentiment words 融入cnn中，instead of hard mapping words.word filter* word,和word_embedding 相似，输入越大，除了能抓取sentiment word net里面的senti words,还能抓到和sentiment word相似的senti words.因为filter 基于Word embedding，做convolution的时候拿到sentiment words和input words,similarity
* reference 对应会议 reference style
* 记录paper怎么做的，和limitation，和我们怎么改变的？
* kcnn  filter parameter 少，random 的少